# Week 2 Assignment 
## Implementation Based Assignment
### I3

Ceron Viveros, Manuela  
mace00001@stud.uni-saarland.de


In [40]:
import numpy as np

In [41]:
DETERMINISTIC = True
BOARD_ROWS = 6
BOARD_COLS = 6

actions = ["move", "turnLeft", "turnRight", "pickMarker", "finish"]
rewards = [0.0, -1.0, 1.0]

"""
- F = Free state, there is no Avatar, wall, or marker in the cell.
– M = Marker state, there is only a Marker in the cell.
– W = Wall state, only grid-cells at position (3,2), (1,3), (4,3) and (0,5) can be walls.
– Ax = Avatar state, the Avatar with orientation x {N, S, E, W } is in the cell.
– AMx = Avatar and Market state, both Marker and Avatar with orientation x {N, S, E, W } are in the cell
"""
facts= ["F", "M", "W", "An", "Ae", "As", "Aw", "AMn", "AMe", "AMs", "AMw"]


# Initial avatar coordinate
avatar_pos = (3,1)

START = [["F", "F", "F", "F", "F", "W"],
        ["F", "F", "F", "W", "F", "F"],
        ["F", "M", "F", "F", "F", "F"],
        ["F", "Aw", "W", "F", "F", "F"],
        ["F", "F", "F", "W", "F", "F"],
        ["M", "F", "F", "F", "F", "F"]]

GOAL = [["F", "F", "F", "F", "F", "W"],
        ["F", "F", "F", "W", "F", "F"],
        ["F", "M", "F", "F", "F", "F"],
        ["F", "F", "W", "F", "F", "F"],
        ["F", "F", "F", "W", "F", "F"],
        ["F", "F", "Ae", "F", "F", "F"]]


In [42]:
class State:
    def __init__(self, state=START, avatar =avatar_pos):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.state = state
        self.currentPos = avatar
        self.isEnd = False
        self.markerTaken = False
        self.determine = DETERMINISTIC
        
    def giveReward(self):
        if self.state == GOAL:
            return 2
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 1
    
    # Dynamic for move action
    def moveRules(self, currentPos, nxtPos, ava, nxt_f):        
        # If cell is Ax
        if (ava == "An") or (ava == "Ae") or (ava == "As") or (ava == "Aw"):
            # If moving to  F cell        
            if nxt_f == "F":
                # Next cell becomes Ax
                nxt_f = ava
            
            # If moving to M cell
            if nxt_f == "M":
                # Next cell becomes AMx
                nxt_f = ava[:1] + "M" + ava[1:]
            
            # Current cell becomes F
            ava = "F"           
        
        # If cell is AMx
        elif (ava == "AMn") or (ava == "AMe") or (ava == "AMs") or (ava == "AMw"):
            # If moving to  F cell        
            if (nxt_f == "F"):
                # Next cell becomes Ax
                nxt_f = ava.replace('M', '')
                #if (self.markerTaken == True): #marker is taken but not put down -> next cell only A
                           
            
            # If moving to M cell
            if nxt_f == "M":
                # Next cell becomes AMx
                nxt_f = ava
            # Current cell becomes F
            if (self.markerTaken == True):
                ava = "F"
            else:
                ava = "M"              
        
        # Update state
        self.state[self.currentPos[0]][self.currentPos[1]] = ava
        self.state[nxtPos[0]][nxtPos[1]] = nxt_f
        
        return self.state
        
            
    def nxtState(self, action):
        """
            0 | 1 | 2| 3| 4 | 5| 6|
        0 |    
        1 |
        2 |
        3 |     A 
        4 |
        5 |
        6 |
        """
        if self.determine:
            
            # Receive actions only if the program is not over
            if self.isEnd == False:
                
                # Fact for the current position of Avatar, eg. An, AMw
                ava= self.state[self.currentPos[0]][self.currentPos[1]] 
            
                if action == actions[0]: # move

                    if (ava == "An") or (ava == "AMn"):
                        nxtPos = (self.currentPos[0]-1, self.currentPos[1]) #move north
                    elif ava == "Ae" or (ava == "AMe"):
                        nxtPos = (self.currentPos[0], self.currentPos[1]+1) #move east
                    elif ava == "As" or (ava == "AMs"):
                        nxtPos = (self.currentPos[0]+1, self.currentPos[1]) #move south
                    else:
                        nxtPos = (self.currentPos[0], self.currentPos[1]-1) #move west                   

                    print("currentPos: ", self.currentPos)
                    print("nxtPos: ", nxtPos)

                    # Ilegal next state
                    if (nxtPos[0] < 0) or (nxtPos[0] >= BOARD_ROWS):
                        self.isEnd = True
                        print("Avatar Crashes")
                        return self.state
                    elif (nxtPos[1] < 0) or (nxtPos[1] >= BOARD_COLS):
                        self.isEnd = True
                        print("Avatar Crashes")
                        return self.state

                    nxt_f = self.state[nxtPos[0]][nxtPos[1]] # Current type next cell

                    # Ilegal next state
                    if nxt_f == "W":        
                        self.isEnd = True
                        print("Avatar Crashes")
                        return self.state

                    newState = self.moveRules(self.currentPos, nxtPos, ava, nxt_f)

                    # Update avatar position and environment state
                    self.currentPos = nxtPos
                    self.state = newState


                elif action == actions[1]: # turnLeft

                    if (ava == "An") or (ava == "AMn"):
                        ava = ava.replace('n','w') #eg. An -> Aw
                    elif (ava == "Aw") or (ava == "AMw"):
                        ava = ava.replace('w','s') 
                    elif (ava == "As") or (ava == "AMs"):
                        ava = ava.replace('s','e') 
                    elif (ava == "Ae") or (ava == "AMe"):
                        ava = ava.replace('e','n')
                        
                    # Update avatar orientation
                    self.state[self.currentPos[0]][self.currentPos[1]] = ava

                elif action == actions[2]: # turnRight

                    if (ava == "An") or (ava == "AMn"):
                        ava = ava.replace('n','e') #eg. AMn -> Ae
                    elif (ava == "Ae") or (ava == "AMe"):
                        ava = ava.replace('e','s')
                    elif (ava == "As") or (ava == "AMs"):
                        ava = ava.replace('s','w')
                    elif (ava == "Aw") or (ava == "AMw"):
                        ava = ava.replace('w','n')

                    # Update avatar orientation
                    self.state[self.currentPos[0]][self.currentPos[1]] = ava

                elif action == actions[3]: # pickMarker
                    # If Marker is present
                    if (ava == "M") or (ava == "AMn") or (ava == "AMe") or (ava == "AMs") or (ava == "AMw"):
                        self.markerTaken = True
                    else:
                        self.isEnd = True
                        print("Avatar Crashes")
                        return self.state
                
                else: # finish
                    self.isEnd = True
                    return self.state
            else:
                print("Program terminated")
            return self.state

In [43]:
c1 = State()

c1.nxtState('move')
c1.nxtState('turnLeft')
c1.nxtState('move')
c1.nxtState('move')
c1.nxtState('pickMarker')
c1.nxtState('turnLeft')
c1.nxtState('move')
c1.nxtState('move')
c1.nxtState('finish')

if (c1.state == GOAL) and (c1.isEnd == True) and (c1.markerTaken == True):
    print("Goal reached")

currentPos:  (3, 1)
nxtPos:  (3, 0)
currentPos:  (3, 0)
nxtPos:  (4, 0)
currentPos:  (4, 0)
nxtPos:  (5, 0)
currentPos:  (5, 0)
nxtPos:  (5, 1)
currentPos:  (5, 1)
nxtPos:  (5, 2)
Goal reached


In [44]:
c2 = State()
c2.nxtState('move')
c2.nxtState('move')

currentPos:  (3, 1)
nxtPos:  (3, 0)
currentPos:  (3, 0)
nxtPos:  (3, -1)
Avatar Crashes


[['F', 'F', 'F', 'F', 'F', 'W'],
 ['F', 'F', 'F', 'W', 'F', 'F'],
 ['F', 'M', 'F', 'F', 'F', 'F'],
 ['F', 'F', 'W', 'F', 'F', 'F'],
 ['F', 'F', 'F', 'W', 'F', 'F'],
 ['F', 'F', 'Ae', 'F', 'F', 'F']]